In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os 
import sys

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# import the regressors
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# import the metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_log_error as MSLE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import median_absolute_error as MAD

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE

# import standard scaler 
from sklearn.preprocessing import StandardScaler

In [3]:
data = 'Data/final_modeling.csv'

In [4]:
df = pd.read_csv(data)

In [5]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,15000,2013.0,ford,others,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,truck,black
1,27990,2012.0,gmc,others,good,8 cylinders,gas,68696.0,clean,other,4wd,truck,black
2,34590,2016.0,chevrolet,others,good,6 cylinders,gas,29499.0,clean,other,4wd,truck,silver
3,35000,2019.0,toyota,tacoma,excellent,6 cylinders,gas,43000.0,clean,automatic,4wd,truck,grey
4,29990,2016.0,chevrolet,others,good,6 cylinders,gas,17302.0,clean,other,4wd,truck,red


In [6]:
df['type'] = df['type'].replace('pickup', 'truck')

In [7]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,15000,2013.0,ford,others,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,truck,black
1,27990,2012.0,gmc,others,good,8 cylinders,gas,68696.0,clean,other,4wd,truck,black
2,34590,2016.0,chevrolet,others,good,6 cylinders,gas,29499.0,clean,other,4wd,truck,silver
3,35000,2019.0,toyota,tacoma,excellent,6 cylinders,gas,43000.0,clean,automatic,4wd,truck,grey
4,29990,2016.0,chevrolet,others,good,6 cylinders,gas,17302.0,clean,other,4wd,truck,red


In [8]:
# calculate age of car
df['age'] = 2021 - df['year']
df['age'] = df['age'].apply(lambda x: x if x > 0 else 0)

# drop year
df = df.drop('year', axis=1)

df.head()

,price,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,age
0,15000,ford,others,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,truck,black,8.0
1,27990,gmc,others,good,8 cylinders,gas,68696.0,clean,other,4wd,truck,black,9.0
2,34590,chevrolet,others,good,6 cylinders,gas,29499.0,clean,other,4wd,truck,silver,5.0
3,35000,toyota,tacoma,excellent,6 cylinders,gas,43000.0,clean,automatic,4wd,truck,grey,2.0
4,29990,chevrolet,others,good,6 cylinders,gas,17302.0,clean,other,4wd,truck,red,5.0


In [9]:
df.shape

(106657, 13)

In [10]:
# Split data into holdout and training sets

# holdout set from 105657 to the end
holdout = df.loc[105658:, :]
print(holdout.shape)

# training set from 0 to 105657
train = df.loc[0:105657, :]
print(train.shape)

(999, 13)
(105658, 13)


In [11]:
df = train

#### Modeling 

In [12]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

import pickle

import os 
import sys


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

from sklearn.feature_selection import SelectPercentile, chi2


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
# import dropout
from keras.layers import Dropout

# import r2 score
from keras import backend as K
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


# import LeakyReLU
from keras.layers import LeakyReLU

from keras.layers import ELU
from keras.layers import ReLU

# import early stopping
from keras.callbacks import EarlyStopping

# import stochastic gradient descent
from keras.optimizers import SGD

In [13]:
numeric_features = ['odometer', 'age']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_features = ['manufacturer', 'model', 'fuel',\
                        'title_status', 'transmission', 'drive', 'type', 'paint_color']
categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder()), 
    ('selector', SelectPercentile(chi2, percentile=15))])

ordinal_features = ['condition', 'cylinders']
ordinal_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder()),
    ('selector', SelectPercentile(chi2, percentile=15))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)])

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # X_train = preprocessor.fit_transform(X_train)
# # X_test = preprocessor.transform(X_test)
# y_train = np.log(y_train)
# y_test = np.log(y_test)

# get input shape
input_shape = X_train.shape[1]

# model 

Regressor = tf.keras.Sequential([
    tf.keras.layers.Dense(100, input_dim=5, kernel_initializer='normal', activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50,  activation='relu'),
    tf.keras.layers.Dense(30, activation='LeakyReLU'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(12, activation='LeakyReLU'),
    tf.keras.layers.Dense(1, activation='linear')
])
Regressor.compile(loss='mean_squared_error', optimizer='adam', metrics=r2_score)
Regressor.summary()


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', Regressor)])

# pipeline.fit(X_train, y_train)
from sklearn import set_config
set_config(display='diagram')
pipeline


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               600       
                                                                 
 dense_1 (Dense)             (None, 500)               50500     
                                                                 
 dense_2 (Dense)             (None, 300)               150300    
                                                                 
 dense_3 (Dense)             (None, 200)               60200     
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_4 (Dense)             (None, 50)                10050     
                                         

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['odometer', 'age']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder()),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=15,
                                                                                    score_func=<function chi2 at 0x2829ef700>))]),
                                                  ['manufacturer', 'model',
                                                   'fuel', 'title_status',
                                                   'transmission', 'drive',
                                                   'type', 'paint_color']),
                                                 ('ord',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder()),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=15,
                                                                                    score_func=<function chi2 at 0x2829ef700>))]),
                                                  ['condition',
                                                   'cylinders'])])),
                ('model',
                 <keras.engine.sequential.Sequential object at 0x2a3c9d730>)])

In [14]:
# fit the model
history = pipeline.fit(X_train, y_train, model__epochs=300, model__batch_size=100, model__verbose=1, model__validation_split=0.2)

Epoch 1/300


2023-06-17 22:51:51.424515: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


677/677 [==============================] - 7s 10ms/step - loss: 81522416.0000 - r2_score: 0.3597 - val_loss: 60098768.0000 - val_r2_score: 0.5265
Epoch 2/300
677/677 [==============================] - 6s 9ms/step - loss: 60362848.0000 - r2_score: 0.5247 - val_loss: 59336432.0000 - val_r2_score: 0.5332
Epoch 3/300
677/677 [==============================] - 6s 9ms/step - loss: 59435704.0000 - r2_score: 0.5331 - val_loss: 59563516.0000 - val_r2_score: 0.5340
Epoch 4/300
677/677 [==============================] - 6s 9ms/step - loss: 59087600.0000 - r2_score: 0.5334 - val_loss: 63475644.0000 - val_r2_score: 0.5052
Epoch 5/300
677/677 [==============================] - 6s 9ms/step - loss: 58916396.0000 - r2_score: 0.5356 - val_loss: 58478516.0000 - val_r2_score: 0.5396
Epoch 6/300
677/677 [==============================] - 6s 9ms/step - loss: 58525056.0000 - r2_score: 0.5383 - val_loss: 58307464.0000 - val_r2_score: 0.5420
Epoch 7/300
677/677 [==============================] - 6s 9ms/step - 

In [16]:
# predictions
y_pred = pipeline.predict(X_test)
print(y_pred)

661/661 [==============================] - 2s 3ms/step
[[27153.21  ]
 [ 7389.7764]
 [21199.932 ]
 ...
 [ 6197.5444]
 [11036.174 ]
 [29395.242 ]]


In [17]:
# evaluate the model
mse = mean_squared_error(y_test, y_pred)

mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

Mean Absolute Error: 5036.635180743201


In [ ]:
import pickle

# save the model to disk
filename = 'DNN_model.pkl'
>> pickle.dump(pipeline, open(filename, 'wb'))